# Task_1: Preparing the dataset.

This first task of the protocol is aimed at preparing a working dataset in PLINK v1.9 binary format with all SNPs identified by the rs number and coordinates based on the genome build GRCh38/hg38; as required by the TopMed imputation server. The working dataset used to illustrate this GWAS protocol is a subset of the HapMap population.

Unmapped and uncertain location variants will be removed.

Run this template if you need to update the build from GRCh37/hg19 to GRCh38/hg38; otherwise you can skip this and go to task 2.  

*These steps are much quicker in bash than in R, even though some steps can take a while as the input dataset is usually big.*

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#UCSC--LiftOver.-Change-the-genomic-assembly-to-build-hg19/GRCh37-(requiered-for-imputation-at-Michigan-Server)" data-toc-modified-id="UCSC--LiftOver.-Change-the-genomic-assembly-to-build-hg19/GRCh37-(requiered-for-imputation-at-Michigan-Server)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>UCSC  LiftOver. Change the genomic assembly to build hg19/GRCh37 (requiered for imputation at Michigan Server)</a></span></li><li><span><a href="#Update-the-database" data-toc-modified-id="Update-the-database-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Update the database</a></span></li><li><span><a href="#Check-for-duplicates" data-toc-modified-id="Check-for-duplicates-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Check for duplicates</a></span></li><li><span><a href="#Update-to-rs-to-obtain-Final-DB-for-QC" data-toc-modified-id="Update-to-rs-to-obtain-Final-DB-for-QC-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Update to rs to obtain Final DB for QC</a></span></li></ul></div>

In [1]:
%load_ext rpy2.ipython

In [2]:
%env Path=/mnt/data/GWAS/output/build38/task1_preQC

env: Path=/mnt/data/GWAS/output/build38/task1_preQC


## UCSC  LiftOver. Change the genomic assembly to build GRCh38/hg38 (requiered for imputation at TopMed Imputation Server)

In [30]:
%%bash
# Update b37 to b38
head /mnt/data/GWAS/input/dataset.b37.pruned.bim

1	1:10107	0	10107	T	TA
1	1:10177	0	10177	AC	A
1	1:10350	0	10350	T	A
1	1:10355	0	10355	G	A
1	1:10462	0	10462	GT	G
1	1:11008	0	11008	G	C
1	1:13110	0	13110	A	G
1	rs201725126	0	13116	G	T
1	1:13273	0	13273	C	G
1	1:14599	0	14599	A	T


In [9]:
%%bash
awk '{OFS="\t"; print "chr"$1,$4,$4+1,$2}' /mnt/data/GWAS/input/dataset.b37.pruned.bim | sed 's/chr23/chrX/g' | sed 's/chr24/chrY/g' | sed 's/chr26/chrM/g' > $Path/UCSC_b37.bed
head $Path/UCSC_b37.bed

chr1	10107	10108	1:10107
chr1	10177	10178	1:10177
chr1	10350	10351	1:10350
chr1	10355	10356	1:10355
chr1	10462	10463	1:10462
chr1	11008	11009	1:11008
chr1	13110	13111	1:13110
chr1	13116	13117	rs201725126
chr1	13273	13274	1:13273
chr1	14599	14600	1:14599


In [10]:
%%bash
# UCSC liftOver to get the same reference build. It creates two output files, hglft_genome.bed and unmapped.bed
# 1st parameter: bed file
# 2nd parameter: fixed path - do not change it
# 3rd parameter: path to output file
# 4th parameter: unmapped SNPs
/mnt/data/GWAS/tools/liftOver $Path/UCSC_b37.bed /mnt/data/GWAS/ref_files/build38/hg19ToHg38.over.chain.gz $Path/hglft_genome.bed $Path/unmapped.bed

Reading liftover chains
Mapping coordinates


In [7]:
%%bash
head $Path/hglft_genome.bed

chr1	10107	10108	1:10107
chr1	10177	10178	1:10177
chr1	10350	10351	1:10350
chr1	10355	10356	1:10355
chr1	10462	10463	1:10462
chr1	11008	11009	1:11008
chr1	13110	13111	1:13110
chr1	13116	13117	rs201725126
chr1	13273	13274	1:13273
chr1	14599	14600	1:14599


##  Update the database

Update the chr, the basepair and the name for each SNP.

In [13]:
%%bash
awk '{OFS="\t"; print $4,$1}' $Path/hglft_genome.bed | sed 's/chrX/chr23/g' | sed 's/chrY/chr24/g' | sed 's/chrM/chr26/g'| sed 's/chr//g'  > $Path/update_chr.txt
awk '{OFS="\t"; print $4,$2}' $Path/hglft_genome.bed > $Path/update_bp.txt
awk '{OFS="\t"; print $4,$1":"$2}' $Path/hglft_genome.bed | sed 's/chr//g' > $Path/update_name.txt


Prepare the unmapped SNPs list to PLINK

In [20]:
%%bash
awk '{OFS="\t"; print $4}' $Path/unmapped.bed | sed '/^$/d' > $Path/rs_to_exclude
head $Path/rs_to_exclude
wc $Path/rs_to_exclude

1:12611561
rs76760850
rs70987258
rs113855685
rs34558133
1:12817301
rs141509525
rs56659575
rs34493018
rs200938605
  928   928 10514 /mnt/data/GWAS/output/build38/task1_preQC/rs_to_exclude


Start prunning with PLINK. Remove Unmapped SNP. Update position (--update-map chr and bp). 

In [24]:
%%bash
# Find plink logs in the Jupyter File view, in this working directory
/usr/lib/plink1.9/plink --bfile /mnt/data/GWAS/input/dataset.b37.pruned --exclude $Path/rs_to_exclude --update-chr $Path/update_chr.txt --allow-extra-chr --make-bed --zero-cms --out $Path/temp1
/usr/lib/plink1.9/plink --bfile $Path/temp1 --update-map $Path/update_bp.txt --allow-extra-chr  --make-bed --out $Path/temp2
/usr/lib/plink1.9/plink --bfile $Path/temp2 --update-name $Path/update_name.txt --allow-extra-chr --chr 1-26 --make-bed --out $Path/temp3

PLINK v1.90b3.45 64-bit (13 Jan 2017)      https://www.cog-genomics.org/plink2
(C) 2005-2017 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /mnt/data/GWAS/output/build38/task1_preQC/temp1.log.
Options in effect:
  --allow-extra-chr
  --bfile /mnt/data/GWAS/input/dataset.b37.pruned
  --exclude /mnt/data/GWAS/output/build38/task1_preQC/rs_to_exclude
  --make-bed
  --out /mnt/data/GWAS/output/build38/task1_preQC/temp1
  --update-chr /mnt/data/GWAS/output/build38/task1_preQC/update_chr.txt
  --zero-cms

36413 MB RAM detected; reserving 18206 MB for main workspace.
1022685 variants loaded from .bim file.
504 people (241 males, 263 females) loaded from .fam.
504 phenotype values loaded from .fam.
--exclude: 1021757 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 504 founders and 0 nonfounders present.
Calculating allele frequencies... 1011121314151617181920212223242526272829303132333435363738394041424344454

/mnt/data/GWAS/output/build38/task1_preQC/temp1.hh ); many commands treat these
as missing.
treat these as missing.
/mnt/data/GWAS/output/build38/task1_preQC/temp2.hh ); many commands treat these
as missing.
treat these as missing.
/mnt/data/GWAS/output/build38/task1_preQC/temp3.hh ); many commands treat these
as missing.
treat these as missing.


##  Check for duplicates

In [25]:
%%bash
#  Remove SNP duplicates
sed 's/ /\t/g' $Path/temp3.bim  | awk '{print $2}' | sort | uniq -c| awk '{if($1>1) print $2}'> $Path/remove_duplicates.txt

In [26]:
%%bash
/usr/lib/plink1.9/plink --bfile $Path/temp3 --exclude $Path/remove_duplicates.txt --make-bed --out $Path/temp4

PLINK v1.90b3.45 64-bit (13 Jan 2017)      https://www.cog-genomics.org/plink2
(C) 2005-2017 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /mnt/data/GWAS/output/build38/task1_preQC/temp4.log.
Options in effect:
  --bfile /mnt/data/GWAS/output/build38/task1_preQC/temp3
  --exclude /mnt/data/GWAS/output/build38/task1_preQC/remove_duplicates.txt
  --make-bed
  --out /mnt/data/GWAS/output/build38/task1_preQC/temp4

36413 MB RAM detected; reserving 18206 MB for main workspace.
1021014 variants loaded from .bim file.
504 people (241 males, 263 females) loaded from .fam.
504 phenotype values loaded from .fam.
--exclude: 1020992 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 504 founders and 0 nonfounders present.
Calculating allele frequencies... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990

/mnt/data/GWAS/output/build38/task1_preQC/temp4.hh ); many commands treat these
as missing.
treat these as missing.


## Update to rs to obtain Final DB for QC


In [29]:
%%bash
/usr/lib/plink1.9/plink --bfile $Path/temp4 --update-name /mnt/data/GWAS/ref_files/build38/All.dbSNP151.chr.bp.b38.rs.update.name.unique.woalleles.txt --make-bed --out $Path/dataset.b38
rm $Path/temp*

PLINK v1.90b3.45 64-bit (13 Jan 2017)      https://www.cog-genomics.org/plink2
(C) 2005-2017 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /mnt/data/GWAS/output/build38/task1_preQC/dataset.b38.log.
Options in effect:
  --bfile /mnt/data/GWAS/output/build38/task1_preQC/temp4
  --make-bed
  --out /mnt/data/GWAS/output/build38/task1_preQC/dataset.b38
  --update-name /mnt/data/GWAS/ref_files/build38/All.dbSNP151.chr.bp.b38.rs.update.name.unique.woalleles.txt

36413 MB RAM detected; reserving 18206 MB for main workspace.
1020992 variants loaded from .bim file.
504 people (241 males, 263 females) loaded from .fam.
504 phenotype values loaded from .fam.
--update-name: 834894 values updated, 617269376 variant IDs not present.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 504 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505

/mnt/data/GWAS/output/build38/task1_preQC/dataset.b38.hh ); many commands treat
these as missing.
treat these as missing.


In [31]:
%%bash
head $Path/dataset.b38.bim

1	1:10107	0	10107	T	TA
1	1:10177	0	10177	AC	A
1	1:10350	0	10350	T	A
1	1:10355	0	10355	G	A
1	rs1200541360	0	10462	GT	G
1	rs575272151	0	11008	G	C
1	rs540538026	0	13110	A	G
1	rs62635286	0	13116	G	T
1	rs531730856	0	13273	C	G
1	rs707680	0	14599	A	T
